# vcc792_bin.ipynb

### Isaac Cheng - September 2021

Various plots of VCC 792 with binning. Read [this webpage](http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/community/ngvs/docs/ngvsdoc.html) for lots of useful information! Also look at [this PDF](https://www.ucolick.org/~bolte/AY257/s_n.pdf) for a useful overview of CCD signal/noise satistics. Also read the section on binning.

Also: [Astropy Course](https://www.iasf-milano.inaf.it/astropy-course/), in particular, see the [Astropy image walkthrough](https://www.iasf-milano.inaf.it/AstropyCourse/07.Images.slides.html).

[vorbin manual](https://www-astro.physics.ox.ac.uk/~mxc/software/vorbin_manual.pdf) and [Michele Cappellari's Python Programs page](http://www-astro.physics.ox.ac.uk/~mxc/software/)

### ~~Current~~ Goals

- Display voronoi binned data without using built-in function?
  - For major axis profile
  - To see pixelation
  - Likely will need to adapt `vorbin` to return more variables.
- Use RA & Dec to display coordinates of binned images


In [1]:
# # May need to do this if running remote server with VS Code
%cd "/arc/home/IsaacCheng/coop_f2021/warmup"

/arc/home/IsaacCheng/coop_f2021/warmup


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
from astropy.wcs import WCS
from astropy.nddata import block_reduce
import vorbin
from vorbin.voronoi_2d_binning import voronoi_2d_binning
import plotbin
import fits_plot_utils as utils
import reproject
from reproject import reproject_interp, reproject_exact
import dill
import pandas as pd
%matplotlib widget

# Voronoi Binning


In [ ]:
# 
# Load data
# 
vcc792_dist = 16.5 * u.Mpc
# Signal
iband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits"
gband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits"
uband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits"
# Noise (sigma maps)
inoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.sig.3136_8588_6905_10184.fits"
gnoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.sig.3136_8588_6905_10184.fits"
unoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.sig.3136_8588_6905_10184.fits"
# Mask (flap maps)
igood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.flag.3136_8588_6905_10184.fits"
ggood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.flag.3136_8588_6905_10184.fits"
ugood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.flag.3136_8588_6905_10184.fits"
# 
# Load NGVS data
# 
# Signal
data_i, header_i = utils.load_img(iband_path)
data_g, header_g = utils.load_img(gband_path)
data_u, header_u = utils.load_img(uband_path)
# Noise
noise_i, header_noise_i = utils.load_img(inoise_path)
noise_g, header_noise_g = utils.load_img(gnoise_path)
noise_u, header_noise_u = utils.load_img(unoise_path)
# Masks
good_i, header_good_i = utils.load_img(igood_path, idx=1)
good_g, header_good_g = utils.load_img(ggood_path, idx=1)
good_u, header_good_u = utils.load_img(ugood_path, idx=1)
# 
# Cut out galaxy
# 
vcc792_center = (2716, 1613)
vcc792_shape = (1480, 1127)
data_i_cut, wcs_i_cut = utils.cutout(data_i, vcc792_center, vcc792_shape, header=header_i)
data_g_cut, wcs_g_cut = utils.cutout(data_g, vcc792_center, vcc792_shape, header=header_g)
data_u_cut, wcs_u_cut = utils.cutout(data_u, vcc792_center, vcc792_shape, header=header_u)
noise_i_cut, wcs_noise_i_cut = utils.cutout(noise_i, vcc792_center, vcc792_shape, header=header_noise_i)
noise_g_cut, wcs_noise_g_cut = utils.cutout(noise_g, vcc792_center, vcc792_shape, header=header_noise_g)
noise_u_cut, wcs_noise_u_cut = utils.cutout(noise_u, vcc792_center, vcc792_shape, header=header_noise_u)
# 
# VERTICO CO Data
# 
# co_path = "/arc/projects/vertico/products/release.v1.2/native/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_mom0_Msun.fits"
co_path = "/arc/projects/vertico/products/release.v1.2/nyquistPix/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_round_k_nyquistPix_9_arcsec_mom0_Msun.fits"
data_co, header_co = utils.load_img(co_path)

In [ ]:
def calc_colour_err(blue, red, blue_err, red_err):
    """
    Calculates the uncertainty in the colour index using basic uncertainty propagation.
    The colour, or colour index, is defined as:
                        colour = -2.5 * log10(blue / red)
    where blue is the flux in the shorter wavelength band and red is the flux in the
    longer wavelength band.
    The uncertainty, assuming independent errors and to a first-order approximation, is
    given by:
            colour_err^2 = (-2.5/ln(10) * 1/blue)^2 * blue_err^2 +
                           (+2.5/ln(10) * 1/red)^2 * red_err^2
                         = (2.5/ln(10))^2 * [(blue_err/blue)^2 + (red_err/red)^2]
    Thus:
            colour_err = sqrt(colour_err^2)
                       = 2.5 / ln(10) * sqrt((blue_err/blue)^2 + (red_err/red)^2)
    Note that all the parameters MUST be able to broadcast together.

    Parameters:
      blue :: array
        The flux in the shorter wavelength band
      red :: array
        The flux in the longer wavelength band
      blue_err :: array
        The uncertainty in the flux in the shorter wavelength band
      red_err :: array
        The uncertainty in the flux in the longer wavelength band
        
    Returns: colour_err
      colour_err :: array
        The uncertainty in the colour index.
    """
    prefactor = 2.5 / np.log(10)
    errs = np.sqrt((blue_err / blue) ** 2 + (red_err / red) ** 2)
    return prefactor * errs


def prelim_bin(signal, noise, block_size=(4, 4), print_info=True, func=np.nansum):
    """
    Regular preliminary binning of 2D data (e.g., for Voronoi binning). Ignores NaNs.
    
    (From astropy's block_reduce() documentation): If the data are not perfectly divisible
    by block_size along a given axis, then the data will be trimmed (from the end) along
    that axis.

    TODO: finish docstring
    """
    # Add signal array using arithmetic sum
    signal_binned = block_reduce(signal, block_size=block_size, func=func)
    # Add noise array in quadrature
    noise_binned = noise * noise
    noise_binned = block_reduce(noise_binned, block_size=block_size, func=func)
    noise_binned = np.sqrt(noise_binned)
    # 
    # Generate pixel coordinates
    # 
    y_coords, x_coords = np.meshgrid(
        np.arange(signal_binned.shape[0]), np.arange(signal_binned.shape[1])
    )
    x_coords, y_coords = x_coords.T, y_coords.T
    # 
    # Ensure no infs or NaNs in binned data (for Voronoi binning)
    # 
    is_good = np.isfinite(signal_binned) & np.isfinite(noise_binned)
    # 
    if print_info:
        print("x_coords, y_coords, signal_binned, noise_binned shapes:",
              x_coords.shape, y_coords.shape, signal_binned.shape, noise_binned.shape)
        print("total bad elements (infs/NaNs):", np.sum(~is_good))
    # 
    return signal_binned, noise_binned, x_coords, y_coords, is_good


def optimal_sn(index, signal, noise):
    """
    Signal-to-noise ratio approximation using optimal weighing of pixels.

    See Eq. (3) of Cappellari & Copin (2003):
    https://ui.adsabs.harvard.edu/abs/2003MNRAS.342..345C/abstract
    
    Parameters: (nearly verbatim from Cappellari & Copin's voronoi_2d_binning.py)
      index :: 1D array
        Integer vector of length N containing the indices of
        the spaxels for which the combined S/N has to be returned.
        The indices refer to elements of the vectors signal and noise.
      signal :: 1D array
        Vector of length M>N with the signal of all spaxels.
      noise :: 1D array
        Vector of length M>N with the noise of all spaxels.
    
    Returns: sn
      sn :: 1D array
        Scalar S/N or another quantity that needs to be equalized.
    """
    return np.sqrt(
        np.sum((signal[index] / noise[index]) * (signal[index] / noise[index]))
    )


def calc_mag(flux, flux_err=0.0, zpt=30.0, calc_abs=False, dist=None, dist_err=0.0):
    """
    Calculates the relative or absolute magnitude of an object given its flux.
    
    Parameters:
      flux :: array
        The flux of the pixel
      flux_err :: array (optional, default: 0.0)
        The uncertainty in the flux. Must be able to broadcast with flux array
      zpt :: scalar
        The zero point of the magnitude system
      calc_abs :: bool (optional, default: False)
        If True, returns the absolute magnitude, otherwise returns the relative magnitude.
        Requires that dist is also provided.
      dist :: scalar or array (optional, default: None)
        The distance to the object/pixel in parsecs. Must be able to broadcast with flux
        array
      dist_err :: scalar or array (optional, default: 0.0)
        The uncertainty in the distance. Must be able to broadcast with flux array
    
    Returns: mag, mag_err
      mag :: array
        The magnitude of the pixel
      mag_err :: array
        The uncertainty in the magnitude
    """
    rel_mag = -2.5 * np.log10(flux) + zpt
    rel_mag_err = 2.5 / np.log(10) * abs(flux_err / flux)
    # 
    if calc_abs:
        if dist is None:
            raise ValueError("dist must be provided if calc_abs is True")
        abs_mag = rel_mag - 5 * (np.log10(dist) - 1)
        abs_mag_err = np.sqrt(rel_mag_err ** 2 + (5 / np.log(10) * dist_err / dist) ** 2)
        return abs_mag, abs_mag_err
    # 
    return rel_mag, rel_mag_err


## Debugging Stuff


In [ ]:
# 
# Explicitly setting array values to floats
# 
# Generate 2500 (total, not 2500 x 2500) uniform pixels
x_coords = np.arange(1,51).astype(int)
y_coords = np.arange(1,51).astype(int)
x_coords, y_coords = np.meshgrid(x_coords, y_coords)
gi_colour = np.full(2500, 5.0)
gi_colour_err = np.full(2500, 1.0)
# 
binNum, xNode, yNode, xBar, yBar, sn, nPixels, scale = voronoi_2d_binning(
    y_coords.flatten(),
    x_coords.flatten(),
    gi_colour.flatten(),
    gi_colour_err.flatten(),
    targetSN=10,
    pixelsize=None,
    cvt=False,
    quiet=True,
    wvt=False,
    sn_func=None,
)
cmap = "plasma"
cmap = mpl.cm.get_cmap(cmap)
#
fig, ax = plt.subplots()
img = ax.scatter(
    x_coords.flatten(), y_coords.flatten(), c=(gi_colour/gi_colour_err), cmap=cmap, s=1
)
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of Original Data (all 5.0)")
ax.set_title("Random data for debugging\\ldots")
# ax.grid(False)
ax.set_aspect("equal")
# fig.savefig("imgs/raw_random_data.pdf")
plt.show()

## g-i Colour


In [ ]:
%matplotlib inline
gi_colour = -2.5 * np.log10(data_g_cut / data_i_cut)
gi_colour_err = calc_colour_err(data_g_cut, data_i_cut, noise_g_cut, noise_i_cut)
# 
# Preliminary binning
#
block_size = (4, 4)
gi_colour = block_reduce(gi_colour, block_size=block_size)
gi_colour_err = block_reduce(gi_colour_err, block_size=block_size)
# 
# Generate pixel coordinates (needed for vorbin)
# 
y_coords, x_coords = np.meshgrid(
    np.arange(gi_colour.shape[0]), np.arange(gi_colour.shape[1])
)
x_coords, y_coords = x_coords.T, y_coords.T
is_good = np.isfinite(gi_colour_err) & np.isfinite(gi_colour)
# 
# Printing some info
# 
print("array shapes:",
      x_coords.shape, y_coords.shape, gi_colour.shape, gi_colour_err.shape)
print("sum bad:", np.sum(~is_good))
# 
# Plot data passed into voronoi_2d_binning()
# 
gi_snr = abs(np.ma.masked_where(~is_good, gi_colour / gi_colour_err))
cmap = "plasma"
cmap = mpl.cm.get_cmap(cmap)
#
# fig, ax = plt.subplots(subplot_kw={"projection": wcs_i_cut})  # auto-sets origin="lower"
fig, ax = plt.subplots()
# img = ax.scatter(
#     x_coords.flatten(), y_coords.flatten(), c=(gi_colour/gi_colour_err), cmap=cmap, s=1
# )
img = ax.imshow(gi_snr, cmap=cmap, origin="lower", vmax=90) 
# img = ax.imshow(gi_colour, cmap=cmap, vmin=0.5, vmax=1.6)
# img = ax.imshow(gi_colour_err, cmap=cmap, vmax=np.percentile(gi_colour_err[is_good], 15))
# utils.add_scalebar(ax, wcs_i_cut, vcc792_dist,
#                    scalebar_factor=2, label="2 kpc", color="w", loc="lower right")
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
# cbar.set_label("g-i Colour Index")
# cbar.set_label("SNR (g-i colour / error)")
cbar.set_label("SNR of Data Passed into \\texttt{vorbin}")
#
# Overplot CO contours from VERTICO data
#
# ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
# 
# ax.tick_params(which="both", direction="out")
# ax.set_xlabel("RA (J2000)")
# ax.set_ylabel("Dec (J2000)")
ax.set_title("VCC 792 / NGC 4380: g-i Colour")
ax.grid(False)
ax.set_aspect("equal")
# fig.savefig("imgs/vcc792_g-i_binned_4x4.pdf")
plt.show()
# 
# ! LARGE IMAGE! Voronoi binning takes a long time. See method docstring
#
gi_binNum, gi_xNode, gi_yNode, gi_xBar, gi_yBar, gi_sn, gi_nPixels, gi_scale = voronoi_2d_binning(
    x_coords[is_good],
    y_coords[is_good],
    gi_colour[is_good],
    gi_colour_err[is_good],
    targetSN=50,
    pixelsize=1,
    cvt=False,
    quiet=False,
    wvt=False,
    sn_func=optimal_sn,
)


In [ ]:
gi_binNum.max()

## u-band


In [ ]:
%matplotlib inline
# 
# Preliminary binning
#
# 2 x 2 binning: ~2120 seconds (> 35 min!), 4 x 4 binning: ~ 111 seconds
data_u_cut_binned, noise_u_cut_binned, x_coords_u, y_coords_u, is_good_u = prelim_bin(
    data_u_cut, noise_u_cut, block_size=(4, 4), print_info=True
)
# 
# Plot data passed into voronoi_2d_binning()
# 
u_snr = abs(np.ma.masked_where(~is_good_u, data_u_cut_binned / noise_u_cut_binned))
cmap = "plasma"
cmap = mpl.cm.get_cmap(cmap)
#
fig, ax = plt.subplots()
img = ax.imshow(u_snr, cmap=cmap, origin="lower", vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of Data Passed into \\texttt{vorbin}")
ax.set_title("VCC 792 / NGC 4380: u-band Data")
ax.grid(False)
ax.set_aspect("equal")
fig.savefig("imgs/vcc792_u_binned_4x4.pdf")
plt.show()
# 
# Voronoi binning
#
u_binNum, u_xNode, u_yNode, u_xBar, u_yBar, u_sn, u_nPixels, u_scale = voronoi_2d_binning(
    x_coords_u[is_good_u],
    y_coords_u[is_good_u],
    data_u_cut_binned[is_good_u],
    noise_u_cut_binned[is_good_u],
    targetSN=50,
    pixelsize=1,
    cvt=False,
    quiet=False,
    wvt=False,
    sn_func=None,
)


In [ ]:
plt.close("all")

# Binning to VERTICO Nyquist / 9 arcsec Resolution


## Loading data


In [36]:
# 
# Load data
# 
# vcc792_dist = 16.5 * u.Mpc
# Signal
zband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184.fits"
iband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits"
gband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits"
uband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits"
# Noise (sigma maps)
znoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10184.fits"
inoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.sig.3136_8588_6905_10184.fits"
gnoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.sig.3136_8588_6905_10184.fits"
unoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.sig.3136_8588_6905_10184.fits"
# Mask (flap maps)
zgood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.flag.3136_8588_6905_10184.fits"
igood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.flag.3136_8588_6905_10184.fits"
ggood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.flag.3136_8588_6905_10184.fits"
ugood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.flag.3136_8588_6905_10184.fits"
# 
# Load NGVS data
# 
# Signal
data_z, header_z = utils.load_img(zband_path)
data_i, header_i = utils.load_img(iband_path)
data_g, header_g = utils.load_img(gband_path)
data_u, header_u = utils.load_img(uband_path)
# Noise
noise_z, header_noise_z = utils.load_img(znoise_path)
noise_i, header_noise_i = utils.load_img(inoise_path)
noise_g, header_noise_g = utils.load_img(gnoise_path)
noise_u, header_noise_u = utils.load_img(unoise_path)
# Masks
good_z, header_good_z = utils.load_img(zgood_path, idx=1)
good_i, header_good_i = utils.load_img(igood_path, idx=1)
good_g, header_good_g = utils.load_img(ggood_path, idx=1)
good_u, header_good_u = utils.load_img(ugood_path, idx=1)
# 
# VERTICO CO Data
# 
co_type = "nyquist"  # "9as" or "nyquist"
# co_path = "/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_9as_round_mom0_Msolpc-2.fits"  # 9 arcsec
co_path = "/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/nyquistPix/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_9as_np_round_mom0_Msolpc-2.fits"  # Nyquist
data_co, header_co = utils.load_img(co_path)
wcs_co = WCS(header_co)
print("USING", co_type, "CO DATA")

Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      54   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      58   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10

## Useful functions for binning

In [3]:
def calc_colour_err(blue, red, blue_err, red_err):
    """
    Calculates the uncertainty in the colour index using basic uncertainty propagation.
    The colour, or colour index, is defined as:
                        colour = -2.5 * log10(blue / red)
    where blue is the flux in the shorter wavelength band and red is the flux in the
    longer wavelength band.
    The uncertainty, assuming independent errors and to a first-order approximation, is
    given by:
            colour_err^2 = (-2.5/ln(10) * 1/blue)^2 * blue_err^2 +
                           (+2.5/ln(10) * 1/red)^2 * red_err^2
                         = (2.5/ln(10))^2 * [(blue_err/blue)^2 + (red_err/red)^2]
    Thus:
            colour_err = sqrt(colour_err^2)
                       = 2.5 / ln(10) * sqrt((blue_err/blue)^2 + (red_err/red)^2)
    Note that all the parameters MUST be able to broadcast together.

    Parameters:
      blue :: array
        The flux in the shorter wavelength band
      red :: array
        The flux in the longer wavelength band
      blue_err :: array
        The uncertainty in the flux in the shorter wavelength band
      red_err :: array
        The uncertainty in the flux in the longer wavelength band
        
    Returns: colour_err
      colour_err :: array
        The uncertainty in the colour index.
    """
    prefactor = 2.5 / np.log(10)
    errs = np.sqrt((blue_err / blue) ** 2 + (red_err / red) ** 2)
    return prefactor * errs


def prelim_bin(signal, noise, block_size=(4, 4), print_info=True, func=np.nansum):
    """
    Regular preliminary binning of 2D data (e.g., for Voronoi binning). Ignores NaNs.
    
    (From astropy's block_reduce() documentation): If the data are not perfectly divisible
    by block_size along a given axis, then the data will be trimmed (from the end) along
    that axis.

    TODO: finish docstring
    """
    # Add signal array using arithmetic sum
    signal_binned = block_reduce(signal, block_size=block_size, func=func)
    # Add noise array in quadrature
    noise_binned = noise * noise
    noise_binned = block_reduce(noise_binned, block_size=block_size, func=func)
    noise_binned = np.sqrt(noise_binned)
    # 
    # Generate pixel coordinates
    # 
    y_coords, x_coords = np.meshgrid(
        np.arange(signal_binned.shape[0]), np.arange(signal_binned.shape[1])
    )
    x_coords, y_coords = x_coords.T, y_coords.T
    # 
    # Ensure no infs or NaNs in binned data (for Voronoi binning)
    # 
    is_good = np.isfinite(signal_binned) & np.isfinite(noise_binned)
    # 
    if print_info:
        print("x_coords, y_coords, signal_binned, noise_binned shapes:",
              x_coords.shape, y_coords.shape, signal_binned.shape, noise_binned.shape)
        print("total bad elements (infs/NaNs):", np.sum(~is_good))
    # 
    return signal_binned, noise_binned, x_coords, y_coords, is_good


def optimal_sn(index, signal, noise):
    """
    Signal-to-noise ratio approximation using optimal weighing of pixels.

    See Eq. (3) of Cappellari & Copin (2003):
    https://ui.adsabs.harvard.edu/abs/2003MNRAS.342..345C/abstract
    
    Parameters: (nearly verbatim from Cappellari & Copin's voronoi_2d_binning.py)
      index :: 1D array
        Integer vector of length N containing the indices of
        the spaxels for which the combined S/N has to be returned.
        The indices refer to elements of the vectors signal and noise.
      signal :: 1D array
        Vector of length M>N with the signal of all spaxels.
      noise :: 1D array
        Vector of length M>N with the noise of all spaxels.
    
    Returns: sn
      sn :: 1D array
        Scalar S/N or another quantity that needs to be equalized.
    """
    return np.sqrt(
        np.sum((signal[index] / noise[index]) * (signal[index] / noise[index]))
    )


def calc_mag(flux, flux_err=0.0, zpt=30.0, calc_abs=False, dist=None, dist_err=0.0):
    """
    Calculates the relative or absolute magnitude of an object given its flux.
    
    Parameters:
      flux :: array
        The flux of the pixel
      flux_err :: array (optional, default: 0.0)
        The uncertainty in the flux. Must be able to broadcast with flux array
      zpt :: scalar
        The zero point of the magnitude system
      calc_abs :: bool (optional, default: False)
        If True, returns the absolute magnitude, otherwise returns the relative magnitude.
        Requires that dist is also provided.
      dist :: scalar or array (optional, default: None)
        The distance to the object/pixel in parsecs. Must be able to broadcast with flux
        array
      dist_err :: scalar or array (optional, default: 0.0)
        The uncertainty in the distance. Must be able to broadcast with flux array
    
    Returns: mag, mag_err
      mag :: array
        The magnitude of the pixel
      mag_err :: array
        The uncertainty in the magnitude
    """
    rel_mag = -2.5 * np.log10(flux) + zpt
    rel_mag_err = 2.5 / np.log(10) * abs(flux_err / flux)
    # 
    if calc_abs:
        if dist is None:
            raise ValueError("dist must be provided if calc_abs is True")
        abs_mag = rel_mag - 5 * (np.log10(dist) - 1)
        abs_mag_err = np.sqrt(rel_mag_err ** 2 + (5 / np.log(10) * dist_err / dist) ** 2)
        return abs_mag, abs_mag_err
    # 
    return rel_mag, rel_mag_err


In [18]:
def get_reproject_shape_factor(target_arr, input_wcs, target_wcs):
    #
    # Determine reprojection shape and binning factor
    #   N.B. reprojection shape should be as close as possible to the shape of a regular
    #   "cut out" if the input array was cut at the boundaries of the target.
    #
    # Find the coordinates of the target_data's edges, assuming the data are rectangular
    target_bottomleft = target_wcs.pixel_to_world(0, 0)
    target_topright = target_wcs.pixel_to_world(*target_arr.shape)
    # ? Don't know if I am passing the arguments in the right order.
    target_centre = target_wcs.pixel_to_world(
        target_arr.shape[1] / 2, target_arr.shape[0] / 2
    )
    # Map the pixels above to their corresponding pixels in the input array
    input_bottomleft = input_wcs.world_to_pixel(target_bottomleft)
    input_topright = input_wcs.world_to_pixel(target_topright)
    input_centre = input_wcs.world_to_pixel(target_centre)
    # Determine binning/transformation factor
    input_to_target_factor = np.subtract(input_topright, input_bottomleft)
    input_to_target_factor = np.round(
        np.divide(input_to_target_factor, target_arr.shape)
    ).astype(int)
    input_reproject_shape = input_to_target_factor * target_arr.shape
    #
    return input_to_target_factor, input_reproject_shape


def reproj_arr(
    input_arr,
    input_wcs,
    target_wcs,
    input_to_target_factor,
    input_reproject_shape,
    reproject_func=reproject.reproject_exact,
):
    """
    Reproject input array to target array.
    
    target_wcs must support CDELT keyword.
    TODO: add support for CD[#]_[#] keywords.
    
    Requires the reproject package: https://reproject.readthedocs.io/en/stable/
    
    TODO: finish docstring
    """
    #
    # Reproject input array to target array
    #
    wcs_reproject = target_wcs.deepcopy()
    wcs_reproject.wcs.crpix = target_wcs.wcs.crpix * input_to_target_factor
    wcs_reproject.wcs.cdelt = target_wcs.wcs.cdelt / input_to_target_factor
    wcs_reproject.array_shape = input_reproject_shape
    arr_reproject = reproject_func(
        (input_arr, input_wcs),
        wcs_reproject,
        shape_out=input_reproject_shape,
        return_footprint=False,
    )
    return arr_reproject, wcs_reproject


def bin_snr_to_target(
    signal_arr,
    signal_wcs,
    noise_arr,
    noise_wcs,
    target_arr,
    target_wcs,
    reproject_func=reproject.reproject_exact,
    bin_func=np.nansum,
    print_debug=False,
):
    """
    Wrapper for binning an signal & noise arrays to the exact physical dimensions and
    resolution of a target (provided via the target_wcs). The input arrays should already
    be masked (i.e., invalid values should be set to np.nan) and the input arrays should
    entirely contain the target_wcs (i.e., the extent of the target_data).

    Requires the reproject package: https://reproject.readthedocs.io/en/stable/

    target_wcs should support the CDELT keyword.
    TODO: add support for CD[#]_[#] keywords.

    TODO: finish docstring
    """
    if print_debug:
        print("target_wcs:", target_wcs)
        print()
    #
    # Determine reprojection shape and binning factor
    #   N.B. reprojection shape should be as close as possible to the shape of a regular
    #   "cut out" if the input array was cut at the boundaries of the target.
    #
    signal_to_target_factor, signal_reproject_shape = get_reproject_shape_factor(
        target_arr, signal_wcs, target_wcs
    )
    if print_debug:
        print("signal_to_target_factor:", signal_to_target_factor)
        print("signal_reproject_shape:", signal_reproject_shape)
        print()
    noise_to_target_factor, noise_reproject_shape = get_reproject_shape_factor(
        target_arr, noise_wcs, target_wcs
    )
    if np.any(signal_reproject_shape != noise_reproject_shape) or np.any(
        signal_to_target_factor != noise_to_target_factor
    ):
        raise ValueError("Signal and noise arrays must have the same shape and wcs.")
    #
    # Reproject data
    #
    # N.B. signal_wcs_reproject and noise_wcs_reproject should be the same
    signal_reproject, signal_wcs_reproject = reproj_arr(
        signal_arr,
        signal_wcs,
        target_wcs,
        signal_to_target_factor,
        signal_reproject_shape,
        reproject_func=reproject_func,
    )
    if print_debug:
        print("signal_wcs_reproject:", signal_wcs_reproject)
        print()
    noise_reproject, noise_wcs_reproject = reproj_arr(
        noise_arr,
        noise_wcs,
        target_wcs,
        noise_to_target_factor,
        noise_reproject_shape,
        reproject_func=reproject_func,
    )
    #
    # Bin to target resolution
    #
    signal_binned, noise_binned, x_coords, y_coords, is_good = prelim_bin(
        signal_reproject,
        noise_reproject,
        block_size=signal_to_target_factor,
        func=bin_func,
        print_info=print_debug,
    )
    # Modify WCS object
    # ? Not sure if slice argument order is correct
    wcs_binned = signal_wcs_reproject.slice(
        (np.s_[:: signal_to_target_factor[0]], np.s_[:: signal_to_target_factor[1]])
    )
    wcs_binned.wcs.crpix = signal_wcs_reproject.wcs.crpix / signal_to_target_factor
    wcs_binned.wcs.cdelt = signal_wcs_reproject.wcs.cdelt * signal_to_target_factor
    if print_debug:
        print("wcs_binned:", wcs_binned)
    #
    return x_coords, y_coords, signal_binned, noise_binned, is_good, wcs_binned


def dill_data(
    outfile,
    reproject_method,
    dist_pc,
    dist_pc_err,
    x_coords,
    y_coords,
    signal_binned,
    noise_binned,
    abs_mag,
    abs_mag_err_withDistErr,
    abs_mag_err_noDistErr,
    is_good,
    wcs_binned,
    wcs_binned_array_shape,
):
    with open(outfile, "wb") as f:
        dill.dump(
            {
                "note": "Remember to set `wcs_binned.array_shape = wcs_binned_array_shape`",
                "reproject_method": reproject_method,
                "dist_pc": dist_pc,
                "dist_pc_err": dist_pc_err,
                "x_coords": x_coords,
                "y_coords": y_coords,
                "signal_binned": signal_binned,
                "noise_binned": noise_binned,
                "abs_mag": abs_mag,
                "abs_mag_err_withDistErr": abs_mag_err_withDistErr,
                "abs_mag_err_noDistErr": abs_mag_err_noDistErr,
                "is_good": is_good,
                "wcs_binned": wcs_binned,
                "wcs_binned_array_shape": wcs_binned_array_shape,  # this is the "NAXIS" keyword
            },
            f,
        )
    print(f"Pickled {outfile}")


def txt_snr(
    outfile,
    xs,
    ys, 
    u_sig,
    u_noise,
    g_sig,
    g_noise,
    i_sig,
    i_noise,
    z_sig,
    z_noise,
):
    id_arr = np.array(
        [str(x).zfill(2) + str(y).zfill(2) for x, y in zip(xs.flatten(), ys.flatten())]
    )
    df = pd.DataFrame(
        {
            "id": id_arr,
            "z": np.zeros(xs.size),
            "counts_u": u_sig.flatten(),
            "err_u": u_noise.flatten(),
            "counts_g": g_sig.flatten(),
            "err_g": g_noise.flatten(),
            "counts_i": i_sig.flatten(),
            "err_i": i_noise.flatten(),
            "counts_z": z_sig.flatten(),
            "err_z": z_noise.flatten(),
        }
    )
    df.to_csv(path_or_buf=outfile, sep=" ", index=False, header=True)
    print(f"Saved {outfile}")


def txt_mags(
    outfile,
    xs,
    ys, 
    u_mag,
    u_mag_err,
    g_mag,
    g_mag_err,
    i_mag,
    i_mag_err,
    z_mag,
    z_mag_err,
):
    id_arr = np.array(
        [str(x).zfill(2) + str(y).zfill(2) for x, y in zip(xs.flatten(), ys.flatten())]
    )
    df = pd.DataFrame(
        {
            "id": id_arr,
            "z": np.zeros(xs.size),
            "u_mag": u_mag.flatten(),
            "u_mag_err": u_mag_err.flatten(),
            "g_mag": g_mag.flatten(),
            "g_mag_err": g_mag_err.flatten(),
            "i_mag": i_mag.flatten(),
            "i_mag_err": i_mag_err.flatten(),
            "z_mag": z_mag.flatten(),
            "z_mag_err": z_mag_err.flatten(),
        }
    )
    df.to_csv(path_or_buf=outfile, sep=" ", index=False, header=True)
    print(f"Saved {outfile}")


def plot_snr(
    signal,
    noise,
    wcs,
    contour_arr,
    countour_wcs,
    contour_levels=[0, 5, 10],
    vmin=150,
    vmax=1250,
    band="u-band",
):
    snr = abs(signal / noise)
    fig, ax = plt.subplots(subplot_kw={"projection": wcs})
    img = ax.imshow(snr, cmap="plasma", vmin=vmin, vmax=vmax)
    cbar = fig.colorbar(img)
    cbar.ax.tick_params(which="both", direction="out")
    cbar.set_label(f"{band} SNR")
    ax.contour(
        contour_arr,
        transform=ax.get_transform(countour_wcs),
        levels=contour_levels,
        colors="w",
    )
    ax.set_title(f"VCC 792 / NGC 4380: {band} Data Binned")
    ax.set_xlabel("RA (J2000)")
    ax.set_ylabel("Dec (J2000)")
    ax.grid(False)
    ax.set_aspect("equal")
    plt.show()
    
def plot_mags(
    mag,
    mag_err,
    wcs,
    contour_arr,
    countour_wcs,
    contour_levels=[0, 5, 10],
    vmin=None,
    vmax=None,
    band="u-band",
):
    fig, ax = plt.subplots(subplot_kw={"projection": wcs})
    img = ax.imshow(mag, cmap="plasma", vmin=vmin, vmax=vmax)
    cbar = fig.colorbar(img)
    cbar.ax.tick_params(which="both", direction="out")
    cbar.set_label(f"{band} Magnitudes")
    ax.contour(
        contour_arr,
        transform=ax.get_transform(countour_wcs),
        levels=contour_levels,
        colors="w",
    )
    ax.set_title(f"VCC 792 / NGC 4380: {band} Data Binned")
    ax.set_xlabel("RA (J2000)")
    ax.set_ylabel("Dec (J2000)")
    ax.grid(False)
    ax.set_aspect("equal")
    plt.show()
    # 
    fig, ax = plt.subplots(subplot_kw={"projection": wcs})
    img = ax.imshow(mag_err, cmap="plasma", vmin=vmin, vmax=vmax)
    cbar = fig.colorbar(img)
    cbar.ax.tick_params(which="both", direction="out")
    cbar.set_label(f"{band} Magnitude Uncertainties")
    ax.contour(
        contour_arr,
        transform=ax.get_transform(countour_wcs),
        levels=contour_levels,
        colors="w",
    )
    ax.set_title(f"VCC 792 / NGC 4380: {band} Data Binned")
    ax.set_xlabel("RA (J2000)")
    ax.set_ylabel("Dec (J2000)")
    ax.grid(False)
    ax.set_aspect("equal")
    plt.show()

## Process & bin data


In [37]:
#
# Mask the data and noise arrays
#
my_mask = np.zeros_like(data_u).astype(bool)
my_mask[2035:2050, 2690:2705] = True  # mask single star near top of VCC 792
my_mask[1156:1187, 2685:2715] = True  # mask single star near bottom of VCC 792
data_u_masked = np.ma.masked_array(data_u, mask=(good_u == 0) | my_mask).filled(fill_value=np.nan)
noise_u_masked = np.ma.masked_array(noise_u, mask=(good_u == 0) | my_mask).filled(fill_value=np.nan)
data_g_masked = np.ma.masked_array(data_g, mask=(good_g == 0) | my_mask).filled(fill_value=np.nan)
noise_g_masked = np.ma.masked_array(noise_g, mask=(good_g == 0) | my_mask).filled(fill_value=np.nan)
data_i_masked = np.ma.masked_array(data_i, mask=(good_i == 0) | my_mask).filled(fill_value=np.nan)
noise_i_masked = np.ma.masked_array(noise_i, mask=(good_i == 0) | my_mask).filled(fill_value=np.nan)
data_z_masked = np.ma.masked_array(data_z, mask=(good_z == 0) | my_mask).filled(fill_value=np.nan)
noise_z_masked = np.ma.masked_array(noise_z, mask=(good_z == 0) | my_mask).filled(fill_value=np.nan)

In [38]:
reproj_func_used = reproject.reproject_exact
reproj_method_used = f"reproject_exact+masked_star2_{co_type}"
# Distances estimated from https://academic.oup.com/mnras/article/428/3/1880/1060205
# (based off of https://ui.adsabs.harvard.edu/abs/1999MNRAS.304..595G/abstract) &
# https://iopscience.iop.org/article/10.1088/0004-637X/694/1/556
vcc792_dist_pc = 23 * 1e6
vcc792_dist_err_pc = 0.1 * vcc792_dist_pc  # 10% uncertainty

### u-band


In [39]:
plt.close("all")
# Bin NGVS data to VERTICO resolution
(
    u_xs_bin,
    u_ys_bin,
    u_signal_bin,
    u_noise_bin,
    u_isgood_bin,
    u_wcs_bin,
) = bin_snr_to_target(
    data_u_masked,
    WCS(header_u),
    noise_u_masked,
    WCS(header_noise_u),
    data_co,
    wcs_co,
    reproject_func=reproj_func_used,
    bin_func=np.nansum,
    print_debug=False,
)
plot_snr(u_signal_bin, u_noise_bin, u_wcs_bin, data_co, wcs_co, vmin=150, vmax=1250, band="u-band")  # Nyquist
# plot_snr(u_signal_bin, u_noise_bin, u_wcs_bin, data_co, wcs_co, vmin=10, vmax=800, band="u-band")  # 9 arcsec
u_mag_bin, u_mag_err_bin_withDistErr = calc_mag(u_signal_bin, u_noise_bin, zpt=30.0, calc_abs=True, dist=vcc792_dist_pc, dist_err=vcc792_dist_err_pc)
_, u_mag_err_bin_noDistErr = calc_mag(u_signal_bin, u_noise_bin, zpt=30.0, calc_abs=True, dist=vcc792_dist_pc, dist_err=0)
plot_mags(u_mag_bin, u_mag_err_bin_withDistErr, u_wcs_bin, data_co, wcs_co, band="u-band (with dist err)")
plot_mags(_, u_mag_err_bin_noDistErr, u_wcs_bin, data_co, wcs_co, band="u-band (no dist err)")
# Pickle data
u_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
dill_data(u_outfile, reproj_method_used, vcc792_dist_pc, vcc792_dist_err_pc, u_xs_bin, u_ys_bin, u_signal_bin, u_noise_bin, u_mag_bin, u_mag_err_bin_withDistErr, u_mag_err_bin_noDistErr, u_isgood_bin, u_wcs_bin, u_wcs_bin.array_shape)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pickled /arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184_BINNED_reproject_exact+masked_star2_nyquist.pkl


### g-band


In [40]:
plt.close("all")
# Bin NGVS data to VERTICO resolution
(
    g_xs_bin,
    g_ys_bin,
    g_signal_bin,
    g_noise_bin,
    g_isgood_bin,
    g_wcs_bin,
) = bin_snr_to_target(
    data_g_masked,
    WCS(header_g),
    noise_g_masked,
    WCS(header_noise_g),
    data_co,
    wcs_co,
    reproject_func=reproj_func_used,
    bin_func=np.nansum,
    print_debug=False,
)
plot_snr(g_signal_bin, g_noise_bin, g_wcs_bin, data_co, wcs_co, vmin=400, vmax=3000, band="g-band")  # Nyquist
# plot_snr(g_signal_bin, g_noise_bin, g_wcs_bin, data_co, wcs_co, vmin=500, vmax=2000, band="g-band")  # 9 arcsec
g_mag_bin, g_mag_err_bin_withDistErr = calc_mag(g_signal_bin, g_noise_bin, zpt=30.0, calc_abs=True, dist=vcc792_dist_pc, dist_err=vcc792_dist_err_pc)
_, g_mag_err_bin_noDistErr = calc_mag(g_signal_bin, g_noise_bin, zpt=30.0, calc_abs=True, dist=vcc792_dist_pc, dist_err=0)
plot_mags(g_mag_bin, g_mag_err_bin_withDistErr, g_wcs_bin, data_co, wcs_co, band="g-band (with dist err)")
plot_mags(_, g_mag_err_bin_noDistErr, g_wcs_bin, data_co, wcs_co, band="g-band (no dist err)")
# Pickle data
g_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
dill_data(g_outfile, reproj_method_used, vcc792_dist_pc, vcc792_dist_err_pc, g_xs_bin, g_ys_bin, g_signal_bin, g_noise_bin, g_mag_bin, g_mag_err_bin_withDistErr, g_mag_err_bin_noDistErr, g_isgood_bin, g_wcs_bin, g_wcs_bin.array_shape)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pickled /arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184_BINNED_reproject_exact+masked_star2_nyquist.pkl


### i-band


In [41]:
plt.close("all")
# Bin NGVS data to VERTICO resolution
(
    i_xs_bin,
    i_ys_bin,
    i_signal_bin,
    i_noise_bin,
    i_isgood_bin,
    i_wcs_bin,
) = bin_snr_to_target(
    data_i_masked,
    WCS(header_i),
    noise_i_masked,
    WCS(header_noise_i),
    data_co,
    wcs_co,
    reproject_func=reproj_func_used,
    bin_func=np.nansum,
    print_debug=False,
)
plot_snr(i_signal_bin, i_noise_bin, i_wcs_bin, data_co, wcs_co, vmin=300, vmax=3000, band="i-band")  # Nyquist
# plot_snr(i_signal_bin, i_noise_bin, i_wcs_bin, data_co, wcs_co, vmin=300, vmax=2000, band="i-band")  # 9 arcsec
i_mag_bin, i_mag_err_bin_withDistErr = calc_mag(i_signal_bin, i_noise_bin, zpt=30.0, calc_abs=True, dist=vcc792_dist_pc, dist_err=vcc792_dist_err_pc)
_, i_mag_err_bin_noDistErr = calc_mag(i_signal_bin, i_noise_bin, zpt=30.0, calc_abs=True, dist=vcc792_dist_pc, dist_err=0)
plot_mags(i_mag_bin, i_mag_err_bin_withDistErr, i_wcs_bin, data_co, wcs_co, band="i-band (with dist err)")
plot_mags(_, i_mag_err_bin_noDistErr, i_wcs_bin, data_co, wcs_co, band="i-band (no dist err)")
# Pickle data
i_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
dill_data(i_outfile, reproj_method_used, vcc792_dist_pc, vcc792_dist_err_pc, i_xs_bin, i_ys_bin, i_signal_bin, i_noise_bin, i_mag_bin, i_mag_err_bin_withDistErr, i_mag_err_bin_noDistErr, i_isgood_bin, i_wcs_bin, i_wcs_bin.array_shape)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pickled /arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184_BINNED_reproject_exact+masked_star2_nyquist.pkl


### z-band


In [42]:
plt.close("all")
# Bin NGVS data to VERTICO resolution
# z-band
(
    z_xs_bin,
    z_ys_bin,
    z_signal_bin,
    z_noise_bin,
    z_isgood_bin,
    z_wcs_bin,
) = bin_snr_to_target(
    data_z_masked,
    WCS(header_z),
    noise_z_masked,
    WCS(header_noise_z),
    data_co,
    wcs_co,
    reproject_func=reproj_func_used,
    bin_func=np.nansum,
    print_debug=False,
)
plot_snr(z_signal_bin, z_noise_bin, z_wcs_bin, data_co, wcs_co, vmin=200, vmax=2000, band="z-band")  # Nyquist
# plot_snr(z_signal_bin, z_noise_bin, z_wcs_bin, data_co, wcs_co, vmin=200, vmax=1500, band="z-band")  # 9 arcsec
z_mag_bin, z_mag_err_bin_withDistErr = calc_mag(z_signal_bin, z_noise_bin, zpt=30.0, calc_abs=True, dist=vcc792_dist_pc, dist_err=vcc792_dist_err_pc)
_, z_mag_err_bin_noDistErr = calc_mag(z_signal_bin, z_noise_bin, zpt=30.0, calc_abs=True, dist=vcc792_dist_pc, dist_err=0)
plot_mags(z_mag_bin, z_mag_err_bin_withDistErr, z_wcs_bin, data_co, wcs_co, band="z-band (with dist err)")
plot_mags(_, z_mag_err_bin_noDistErr, z_wcs_bin, data_co, wcs_co, band="z-band (no dist err)")
# Pickle data
z_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
dill_data(z_outfile, reproj_method_used, vcc792_dist_pc, vcc792_dist_err_pc, z_xs_bin, z_ys_bin, z_signal_bin, z_noise_bin, z_mag_bin, z_mag_err_bin_withDistErr, z_mag_err_bin_noDistErr, z_isgood_bin, z_wcs_bin, z_wcs_bin.array_shape)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Pickled /arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184_BINNED_reproject_exact+masked_star2_nyquist.pkl


### Export data to text file


In [3]:
# 
# Read pickle files
# 
# with open(u_outfile, "rb") as fu:
    # file = dill.load(fu)
    # print(file.keys())
    # print(file["note"])
    # print(file["reproject_method"])
    # print(file["x_coords"])
    # print(file["y_coords"])
    # print(file["signal_binned"])
    # print(file["noise_binned"])
    # print(file["is_good"])
    # print(file["wcs_binned"])
    # print(file["wcs_binned_array_shape"])
# 
# Generate ASCII txt file
# 
# Load pickled data
co_type = "9as"
reproj_method_used = f"reproject_exact+masked_star2_{co_type}"
# 
u_infile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
with open(u_infile, "rb") as f:
    u_file = dill.load(f)
    u_xs_bin = u_file["x_coords"]
    u_ys_bin = u_file["y_coords"]
    u_mag_bin = u_file["abs_mag"]
    u_mag_err_bin_withDistErr = u_file["abs_mag_err_withDistErr"]
    u_mag_err_bin_noDistErr = u_file["abs_mag_err_noDistErr"]
g_infile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
with open(g_infile, "rb") as f:
    g_file = dill.load(f)
    g_mag_bin = g_file["abs_mag"]
    g_mag_err_bin_withDistErr = g_file["abs_mag_err_withDistErr"]
    g_mag_err_bin_noDistErr = g_file["abs_mag_err_noDistErr"]
i_infile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
with open(u_infile, "rb") as f:
    i_file = dill.load(f)
    i_mag_bin = i_file["abs_mag"]
    i_mag_err_bin_withDistErr = i_file["abs_mag_err_withDistErr"]
    i_mag_err_bin_noDistErr = i_file["abs_mag_err_noDistErr"]
z_infile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184_BINNED_{reproj_method_used}.pkl"
with open(g_infile, "rb") as f:
    z_file = dill.load(f)
    z_mag_bin = z_file["abs_mag"]
    z_mag_err_bin_withDistErr = z_file["abs_mag_err_withDistErr"]
    z_mag_err_bin_noDistErr = z_file["abs_mag_err_noDistErr"]
print(u_xs_bin.max(), u_ys_bin.max())
# 
txt_outfile = f"/arc/home/IsaacCheng/coop_f2021/binned_data/vcc792_BINNED_{reproj_method_used}_noDistErr.txt"
# txt_data(
#     txt_outfile,
#     u_xs_bin,
#     u_ys_bin,
#     u_signal_bin,
#     u_noise_bin,
#     g_signal_bin,
#     g_noise_bin,
#     i_signal_bin,
#     i_noise_bin,
#     z_signal_bin,
#     z_noise_bin,
# )
# txt_mags(
#     txt_outfile,
#     u_xs_bin,
#     u_ys_bin,
#     u_mag_bin,
#     u_mag_err_bin_noDistErr,
#     g_mag_bin,
#     g_mag_err_bin_noDistErr,
#     i_mag_bin,
#     i_mag_err_bin_noDistErr,
#     z_mag_bin,
#     z_mag_err_bin_noDistErr,
# )

79 79


## Old code (for testing, warmup, etc.)


In [ ]:
# 
# Binning NGVS to CO Nyquist resolution. Not "apples to apples"
# 
%matplotlib widget
plt.close("all")
# 
# u-band data
# 
u_snr = abs(data_u / noise_u)
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_u)})
img = ax.imshow(u_snr, cmap="plasma", origin="lower", vmin=0, vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# CO Nyquist-sampled data
# 
data_co_masked = np.ma.masked_where(data_co <= 0, data_co)
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_co)})
img = ax.contourf(data_co_masked, levels=15, cmap="magma_r", vmin=0)
# img = ax.imshow(data_co_masked, cmap="magma_r", vmin=0)
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label(r"Surface Density ($\rm M_\odot\; pc^{-2}$)")
ax.set_title("VCC 792 / NGC 4380: Nyquist-sampled CO Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Print stats
# TODO: Clean up this junk!
# 
# ? Don't know if I am passing the arguments in the right order.
# ? Luckily data_co is symmetric: (42, 42)
print("wcs_co", WCS(header_co))
co_bottomleft = WCS(header_co).pixel_to_world(0, 0)
co_topright = WCS(header_co).pixel_to_world(*data_co.shape)
co_centre = WCS(header_co).pixel_to_world(data_co.shape[1] // 2, data_co.shape[0] // 2)
# print(co_centre)
# print(WCS(header_u).world_to_pixel(co_bottomleft))
# print(WCS(header_u).world_to_pixel(co_topright))
# print(WCS(header_u).world_to_pixel(co_centre))
u_centre_of_co = WCS(header_u).world_to_pixel(co_centre)
u_bottomleft_of_co = WCS(header_u).world_to_pixel(co_bottomleft)
u_topright_of_co = WCS(header_u).world_to_pixel(co_topright)
u_to_co_factor = np.subtract(u_topright_of_co, u_bottomleft_of_co)
u_to_co_factor = np.ceil(np.divide(u_to_co_factor, data_co.shape)).astype(int)
u_shape_of_co = u_to_co_factor * data_co.shape
# print(u_to_co_factor)
# print("u_shape_of_co", u_shape_of_co)
# print(WCS(header_u).pixel_to_world(*u_centre_of_co))
# 
# Cut data
# 
data_u_cut, wcs_u_cut = utils.cutout(data_u, co_centre, u_shape_of_co, header=header_u)
noise_u_cut, wcs_noise_u_cut = utils.cutout(noise_u, co_centre, u_shape_of_co, header=header_noise_u)
u_snr_cut = abs(data_u_cut / noise_u_cut)
fig, ax = plt.subplots(subplot_kw={"projection": wcs_u_cut})
img = ax.imshow(u_snr_cut, cmap="plasma", origin="lower", vmin=0, vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data Cut")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Bin to VERTICO resolution
#
data_u_binned, noise_u_binned, x_coords, y_coords, is_good = prelim_bin(
    data_u_cut, noise_u_cut, block_size=u_to_co_factor
)
# * FIXME: Update WCS 
# ? Not sure if slice argument order is correct. Good thing it is symmetric here...
# https://docs.astropy.org/en/stable/api/astropy.wcs.WCS.html#astropy.wcs.WCS.slice
wcs_u_binned = wcs_u_cut.slice(
    (np.s_[::u_to_co_factor[0]], np.s_[::u_to_co_factor[1]])
)
wcs_u_binned.wcs.crpix = np.round(wcs_u_cut.wcs.crpix / u_to_co_factor)
wcs_u_binned.wcs.cd = wcs_u_cut.wcs.cd * u_to_co_factor
print("wcs_u_cut", wcs_u_cut)
print()
print("wcs_u_binned", wcs_u_binned)
# Plot
u_snr_binned = abs(data_u_binned / noise_u_binned)
fig, ax = plt.subplots(subplot_kw={"projection": wcs_u_binned})
# fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_co)})  # not entirely correct
img = ax.imshow(u_snr_binned, cmap="plasma", origin="lower", vmin=150, vmax=1250) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data Binned")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()

In [ ]:
# 
# Binning NGVS to CO Nyquist / 9 arcsec resolution with reproject. "Apples to apples"
# 
%matplotlib widget
plt.close("all")
# 
# Mask the data
# 
my_mask = np.zeros_like(data_u).astype(bool)
my_mask[2035:2050, 2690:2705] = True  # mask single star near top of VCC 792
my_mask[1156:1187, 2685:2715] = True  # mask single star near bottom of VCC 792
data_u_masked = np.ma.masked_where((good_u == 0) | my_mask, data_u)
data_u_masked[data_u_masked.mask] = np.nan  # explicitly set to NaN
noise_u_masked = np.ma.masked_where((good_u == 0) | my_mask, noise_u)
noise_u_masked[noise_u_masked.mask] = np.nan  # explicitly set to NaN
# 
# u-band data
# 
u_snr = abs(data_u_masked / noise_u_masked)
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_u)})
# fig, ax = plt.subplots()
img = ax.imshow(u_snr, cmap="plasma", origin="lower", vmin=0, vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# CO Nyquist-sampled data
# 
data_co_masked = np.ma.masked_where(data_co <= 0, data_co)
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_co)})
img = ax.contourf(data_co_masked, levels=15, cmap="magma_r", vmin=0)
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='k')
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label(r"Surface Density ($\rm M_\odot\; pc^{-2}$)")
ax.set_title("VCC 792 / NGC 4380: Nyquist-sampled CO Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Print stats
# TODO: Clean up this junk!
# 
# ? Don't know if I am passing the arguments in the right order.
# ? Luckily data_co is symmetric: (42, 42)
# print("wcs_co", wcs_co)
co_bottomleft = wcs_co.pixel_to_world(0, 0)
co_topright = wcs_co.pixel_to_world(*data_co.shape)
co_centre = wcs_co.pixel_to_world(data_co.shape[1] / 2, data_co.shape[0] / 2)
# print("co_centre:", co_centre)
# print(WCS(header_u).world_to_pixel(co_bottomleft))
# print(WCS(header_u).world_to_pixel(co_topright))
# print(WCS(header_u).world_to_pixel(co_centre))
u_centre_of_co = WCS(header_u).world_to_pixel(co_centre)
# print("u_centre_of_co:", u_centre_of_co)
u_bottomleft_of_co = WCS(header_u).world_to_pixel(co_bottomleft)
u_topright_of_co = WCS(header_u).world_to_pixel(co_topright)
u_to_co_factor = np.subtract(u_topright_of_co, u_bottomleft_of_co)
u_to_co_factor_unrounded = np.divide(u_to_co_factor, data_co.shape)
# print("unrounded u_to_co_factor:", u_to_co_factor_unrounded)
u_to_co_factor = np.round(np.divide(u_to_co_factor, data_co.shape)).astype(int)
u_reproject_shape = u_to_co_factor * data_co.shape
# print("u_to_co_factor:", u_to_co_factor)
# print("u_reproject_shape", u_reproject_shape)
# print(WCS(header_u).pixel_to_world(*u_centre_of_co))
# 
# Reproject data
# 
wcs_co_reproject = wcs_co.deepcopy()
wcs_co_reproject.wcs.crpix = wcs_co.wcs.crpix * u_to_co_factor
wcs_co_reproject.wcs.cdelt = wcs_co.wcs.cdelt / u_to_co_factor
wcs_co_reproject.array_shape = u_reproject_shape
# print("wcs_co_reproject", wcs_co_reproject)
u_data_reproj = reproject_interp(
    (data_u_masked, header_u), wcs_co_reproject, shape_out=u_reproject_shape, return_footprint=False
)
u_noise_reproj = reproject_interp(
    (noise_u_masked, header_noise_u), wcs_co_reproject, shape_out=u_reproject_shape, return_footprint=False
)
# print("u_data_reproj.shape", u_data_reproj.shape)
# 
u_snr_reproject = abs(u_data_reproj / u_noise_reproj)
fig, ax = plt.subplots(subplot_kw={"projection": wcs_co_reproject})
img = ax.imshow(u_snr_reproject, cmap="plasma", origin="lower", vmin=0, vmax=90) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of Reprojected u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Reprojected Data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Bin to VERTICO resolution
# #
data_u_binned, noise_u_binned, x_coords, y_coords, is_good = prelim_bin(
    u_data_reproj, u_noise_reproj, block_size=u_to_co_factor, func=np.nansum
)
# ? Not sure if slice argument order is correct. Good thing it is symmetric here...
wcs_u_binned = wcs_co_reproject.slice(
    (np.s_[::u_to_co_factor[0]], np.s_[::u_to_co_factor[1]])
)
wcs_u_binned.wcs.crpix = wcs_co_reproject.wcs.crpix / u_to_co_factor
wcs_u_binned.wcs.cdelt = wcs_co_reproject.wcs.cdelt * u_to_co_factor
# print("wcs_u_reproject", wcs_co_reproject)
# print()
# print("wcs_u_binned", wcs_u_binned)
# Plot
u_snr_binned = abs(data_u_binned / noise_u_binned)
fig, ax = plt.subplots(subplot_kw={"projection": wcs_u_binned})
# img = ax.imshow(u_snr_binned, cmap="plasma", origin="lower", vmin=150, vmax=1250) 
img = ax.imshow(u_snr_binned, cmap="plasma", origin="lower", vmin=100, vmax=800) 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("SNR of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data Binned")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
# Calculate absolute magnitudes
# 
# Assign distance of 23 +/- 1 Mpc for VCC 792.
# https://academic.oup.com/mnras/article/428/3/1880/1060205 &
# https://iopscience.iop.org/article/10.1088/0004-637X/694/1/556
mag_u_binned, mag_u_binned_err = calc_mag(
    data_u_binned, noise_u_binned, zpt=30.0, calc_abs=True, dist=23 * 1e6, dist_err=1e6
)
# 
fig, ax = plt.subplots(subplot_kw={"projection": wcs_u_binned})
img = ax.imshow(mag_u_binned, cmap="plasma") 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("Absolute Magnitude of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data Binned")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()
# 
fig, ax = plt.subplots(subplot_kw={"projection": wcs_u_binned})
img = ax.imshow(mag_u_binned_err, cmap="plasma") 
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label("Uncertainty in Absolute Magnitude of u-Band Data")
ax.contour(data_co, transform=ax.get_transform(WCS(header_co)), levels=0, colors='w')
ax.set_title("VCC 792 / NGC 4380: u-band Data Binned")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()

In [ ]:
plt.close("all")
# Reproject u-band data to CO data
u_data_reproj, u_data_footprint = reproject_exact(
    (data_u, header_u), header_co, parallel=4
)
u_noise_reproj, u_noise_footprint = reproject_exact(
    (noise_u, header_noise_u), header_co, parallel=4
)
u_snr_reproj = abs(u_data_reproj / u_noise_reproj)
# 
# Plot
# 
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw={"projection": WCS(header_co)})
# 
img1 = ax1.imshow(u_snr_reproj, cmap="plasma", origin="lower", vmin=0, vmax=90)
# cbar1 = fig.colorbar(img1)
# cbar1.ax.tick_params(which="both", direction="out")
# cbar1.set_label("SNR of u-Band Data")
# Plot CO contours
ax1.contour(data_co, transform=ax1.get_transform(WCS(header_co)), levels=0, colors='w')
ax1.set_title("u-band Reprojected")
ax1.grid(False)
ax1.set_aspect("equal")
# 
ax2.imshow(u_data_footprint, origin='lower', vmin=0, vmax=1.5)
ax2.set_title("u-band Footprint")
ax2.coords['dec'].set_axislabel_position('r')
ax2.coords['dec'].set_ticklabel_position('r')
ax2.grid(False)
ax2.set_aspect("equal")
fig.suptitle("VCC 792 / NGC 4380")
plt.show()

In [ ]:
plt.close("all")
# 
# Reproject u-band data to CO data
# 
u_data_reproj, u_data_footprint = reproject_interp((data_u, header_u), header_co)
u_noise_reproj, u_noise_footprint = reproject_interp((noise_u, header_noise_u), header_co)
u_snr_reproj = abs(u_data_reproj / u_noise_reproj)
# 
# Plot
# 
fig, ax = plt.subplots(subplot_kw={"projection": WCS(header_co)})
img = ax.imshow(u_snr_reproj, cmap="plasma", origin="lower", vmin=0)
cbar = fig.colorbar(img)
cbar.ax.tick_params(which="both", direction="out")
cbar.set_label(r"SNR of \texttt{reproject\_interp()} u-Band Data")
ax.contour(
    data_co, transform=ax.get_transform(WCS(header_co)), levels=range(0, 15, 5), colors='w'
)
ax.set_title("VCC 792 / NGC 4380: u-band Data Reprojected")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
plt.show()

# High Resolution NGVS data (Voronoi binning?)


## Loading data


In [6]:
# 
# Load data
# 
# vcc792_dist = 16.5 * u.Mpc
# Signal
zband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184.fits"
iband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits"
gband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits"
uband_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits"
# Noise (sigma maps)
znoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10184.fits"
inoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.sig.3136_8588_6905_10184.fits"
gnoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.sig.3136_8588_6905_10184.fits"
unoise_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.sig.3136_8588_6905_10184.fits"
# Mask (flap maps)
zgood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.flag.3136_8588_6905_10184.fits"
igood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.flag.3136_8588_6905_10184.fits"
ggood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.flag.3136_8588_6905_10184.fits"
ugood_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.flag.3136_8588_6905_10184.fits"
# 
# Load NGVS data
# 
# Signal
data_z, header_z = utils.load_img(zband_path)
data_i, header_i = utils.load_img(iband_path)
data_g, header_g = utils.load_img(gband_path)
data_u, header_u = utils.load_img(uband_path)
# Noise
noise_z, header_noise_z = utils.load_img(znoise_path)
noise_i, header_noise_i = utils.load_img(inoise_path)
noise_g, header_noise_g = utils.load_img(gnoise_path)
noise_u, header_noise_u = utils.load_img(unoise_path)
# Masks
good_z, header_good_z = utils.load_img(zgood_path, idx=1)
good_i, header_good_i = utils.load_img(igood_path, idx=1)
good_g, header_good_g = utils.load_img(ggood_path, idx=1)
good_u, header_good_u = utils.load_img(ugood_path, idx=1)
# # 
# # VERTICO CO Data
# # 
co_type = "9as"  # "9as" or "nyquist"
if co_type == "9as":
    co_path = "/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_9as_round_mom0_Msolpc-2.fits"  # 9 arcsec
elif co_type == "nyquist":
    co_path = "/arc/home/IsaacCheng/coop_f2021/vertico_data/v1.3/nyquistPix/9arcsec/NGC4380/NGC4380_7m+tp_co21_pbcorr_9as_np_round_mom0_Msolpc-2.fits"  # Nyquist
else:
    raise ValueError("Invalid co_type. Must be either '9as' or 'nyquist'")
data_co, header_co = utils.load_img(co_path)
wcs_co = WCS(header_co)
print("USING", co_type, "CO DATA")

Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      54   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.u.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      58   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.z.Mg004.sig.3136_8588_6905_10

In [7]:
def cutout_to_target(input_arr, input_wcs, target_arr, target_wcs, mode="trim"):
    """
    Uses Cutout2D to cut out a subsction of input_arr to the extent of target_arr
    (provided via the target_wcs). The extent of the target_arr should be wholly contained
    inside the extent of input_arr.

    Parameters:
      input_arr :: 2D array
        The array from which the cutout will be extracted
      input_wcs :: `astropy.wcs.WCS`
        The WCS of the input_arr
      target_arr :: 2D array
        The array whose extent you want to copy
      target_wcs :: `astropy.wcs.WCS`
        The WCS of the target_arr
    
    Returns: input_arr_cut, input_wcs_cut
      input_data_cut :: 2D array
        The array cut out from the original input_arr
      input_wcs_cut :: `astropy.wcs.WCS` or None
        The updated WCS for the cutout array
    """
    target_bottomleft = target_wcs.pixel_to_world(0, 0)
    target_topright = target_wcs.pixel_to_world(*target_arr.shape)
    # ? Don't know if I am passing the arguments in the right order.
    target_centre = target_wcs.pixel_to_world(
        target_arr.shape[1] / 2, target_arr.shape[0] / 2
    )
    # Map the pixels above to their corresponding pixels in the input array
    input_bottomleft = input_wcs.world_to_pixel(target_bottomleft)
    input_topright = input_wcs.world_to_pixel(target_topright)
    # Determine shape of cutout in input array
    cutout_shape = abs(np.subtract(input_topright, input_bottomleft))
    return utils.cutout(input_arr, target_centre, cutout_shape, wcs=input_wcs, mode=mode)

## Process data


In [8]:
#
# Mask the data and noise arrays
#
my_mask = np.zeros_like(data_u).astype(bool)
my_mask[2035:2050, 2690:2705] = True  # mask single star near top of VCC 792
my_mask[1156:1187, 2685:2715] = True  # mask single star near bottom of VCC 792
data_u_masked = np.ma.masked_array(data_u, mask=(good_u == 0) | my_mask).filled(fill_value=np.nan)
noise_u_masked = np.ma.masked_array(noise_u, mask=(good_u == 0) | my_mask).filled(fill_value=np.nan)
data_g_masked = np.ma.masked_array(data_g, mask=(good_g == 0) | my_mask).filled(fill_value=np.nan)
noise_g_masked = np.ma.masked_array(noise_g, mask=(good_g == 0) | my_mask).filled(fill_value=np.nan)
data_i_masked = np.ma.masked_array(data_i, mask=(good_i == 0) | my_mask).filled(fill_value=np.nan)
noise_i_masked = np.ma.masked_array(noise_i, mask=(good_i == 0) | my_mask).filled(fill_value=np.nan)
data_z_masked = np.ma.masked_array(data_z, mask=(good_z == 0) | my_mask).filled(fill_value=np.nan)
noise_z_masked = np.ma.masked_array(noise_z, mask=(good_z == 0) | my_mask).filled(fill_value=np.nan)
# 
# Cut out galaxy
# 
# vcc792_center = (2716, 1613)
# vcc792_shape = (850, 650)
# data_u_cut, wcs_u_cut = utils.cutout(data_u_masked, vcc792_center, vcc792_shape, header=header_u)
# data_g_cut, wcs_g_cut = utils.cutout(data_g_masked, vcc792_center, vcc792_shape, header=header_g)
# data_i_cut, wcs_i_cut = utils.cutout(data_i_masked, vcc792_center, vcc792_shape, header=header_i)
# data_z_cut, wcs_z_cut = utils.cutout(data_z_masked, vcc792_center, vcc792_shape, header=header_z)
# noise_u_cut, wcs_noise_u_cut = utils.cutout(noise_u_masked, vcc792_center, vcc792_shape, header=header_noise_u)
# noise_g_cut, wcs_noise_g_cut = utils.cutout(noise_g_masked, vcc792_center, vcc792_shape, header=header_noise_g)
# noise_i_cut, wcs_noise_i_cut = utils.cutout(noise_i_masked, vcc792_center, vcc792_shape, header=header_noise_i)
# noise_z_cut, wcs_noise_z_cut = utils.cutout(noise_z_masked, vcc792_center, vcc792_shape, header=header_noise_z)
data_u_cut, wcs_u_cut = cutout_to_target(data_u_masked, WCS(header_u), data_co, wcs_co)
data_g_cut, wcs_g_cut = cutout_to_target(data_g_masked, WCS(header_g), data_co, wcs_co)
data_i_cut, wcs_i_cut = cutout_to_target(data_i_masked, WCS(header_i), data_co, wcs_co)
data_z_cut, wcs_z_cut = cutout_to_target(data_z_masked, WCS(header_z), data_co, wcs_co)
noise_u_cut, wcs_noise_u_cut = cutout_to_target(noise_u_masked, WCS(header_noise_u), data_co, wcs_co)
noise_g_cut, wcs_noise_g_cut = cutout_to_target(noise_g_masked, WCS(header_noise_g), data_co, wcs_co)
noise_i_cut, wcs_noise_i_cut = cutout_to_target(noise_i_masked, WCS(header_noise_i), data_co, wcs_co)
noise_z_cut, wcs_noise_z_cut = cutout_to_target(noise_z_masked, WCS(header_noise_z), data_co, wcs_co)


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


## (Testing) SNR plots


In [9]:
%matplotlib widget
plt.close("all")
print(data_u_cut.shape)
block_size = (3, 3)
u_data_binned, u_noise_binned, _, _, u_isgood = utils.prelim_bin(
    data_u_cut, noise_u_cut, block_size=block_size, print_info=True, func=np.nansum
)
# u_snr = abs(data_u_cut / noise_u_cut)
u_snr = abs(u_data_binned / u_noise_binned)
# fig, ax = plt.subplots(subplot_kw={'projection': wcs_u_cut})
fig, ax = plt.subplots()
u_img = ax.imshow(u_snr, origin="lower", cmap="plasma", vmin=15, vmax=150)
u_cbar = fig.colorbar(u_img, ax=ax)
u_cbar.set_label("SNR")
# ax.contour(
#     data_co, transform=ax.get_transform(wcs_co), levels=range(0, 15, 5), colors='w'
# )
ax.set_title(f"VCC 792: u-band ${block_size[0]} \\times {block_size[1]}$ binned data")
ax.set_xlabel("RA (J2000)")
ax.set_ylabel("Dec (J2000)")
ax.grid(False)
ax.set_aspect("equal")
# fig.savefig(f"imgs/vcc792_u_binned_{block_size[0]}x{block_size[1]}.pdf")
plt.show()
# 
u_snr_masked = np.ma.masked_array(u_snr, mask=u_snr < 30).filled(fill_value=np.nan)
# fig, ax = plt.subplots(subplot_kw={'projection': wcs_u_cut})
fig, ax = plt.subplots()
u_img = ax.imshow(u_snr_masked, origin="lower", cmap="plasma", vmax=180)
u_cbar = fig.colorbar(u_img, ax=ax)
u_cbar.set_label("SNR")
# ax.contour(
#     data_co, transform=ax.get_transform(wcs_co), levels=range(0, 15, 5), colors='w'
# )
ax.set_title(f"VCC 792: u-band ${block_size[0]} \\times {block_size[1]}$ binned data")
ax.set_xlabel("Pixels in RA Direction")
ax.set_ylabel("Pixels in Dec Direction")
ax.grid(False)
ax.set_aspect("equal")
# fig.savefig(f"imgs/vcc792_u_binned_{block_size[0]}x{block_size[1]}.pdf")
plt.show()

(862, 861)
x_coords, y_coords, signal_binned, noise_binned shapes: (287, 287) (287, 287) (287, 287) (287, 287)
total bad elements (infs/NaNs): 0


/tmp/ipykernel_1908/1605387202.py:9: RuntimeWarning: invalid value encountered in true_divide
  u_snr = abs(u_data_binned / u_noise_binned)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Applying same binning pattern across all bands


In [ ]:
snr_target_u = 30
#
y_coords_u, x_coords_u = np.meshgrid(
    np.arange(data_u_cut.shape[0]), np.arange(data_u_cut.shape[1])
)
x_coords_u, y_coords_u = x_coords_u.T, y_coords_u.T
good_vorbin_u = np.isfinite(data_u_cut) & np.isfinite(noise_u_cut)
# 
# Voronoi binning of u-band data
# 
binNum_u, xNode_u, yNode_u, xBar_u, yBar_u, sn_u, nPixels_u, scale_u = voronoi_2d_binning(
    x_coords_u[good_vorbin_u],
    y_coords_u[good_vorbin_u],
    data_u_cut[good_vorbin_u],
    noise_u_cut[good_vorbin_u],
    targetSN=snr_target_u,
    pixelsize=1,
    cvt=False,
    quiet=True,
    wvt=False,
    sn_func=None,
)
# 
# Save data
# 
outfile_u = "/arc/home/IsaacCheng/coop_f2021/warmup/vcc792_u_snr30.pkl"
with open(outfile_u, "wb") as f:
    dill.dump(
        {
            # "full_shape": np.shape(data_u_cut),  # same as good_vorbin.shape
            "x_coords": x_coords_u,
            "y_coords": y_coords_u,
            "signal": data_u_cut,
            "noise": noise_u_cut,
            "good_vorbin": good_vorbin_u,
            "snr_target": snr_target_u,
            "binNum": binNum_u,
            "xNode": xNode_u,
            "yNode": yNode_u,
            "xBar": xBar_u,
            "yBar": yBar_u,
            "sn": sn_u,
            "nPixels": nPixels_u,
            "scale": scale_u,
        },
        f,
    )
print(f"Pickled {outfile_u}")

In [131]:
%matplotlib widget
plt.close("all")
outfile_u = "/arc/home/IsaacCheng/coop_f2021/warmup/vcc792_u_snr30.pkl"
with open(outfile_u, "rb") as f:
    file = dill.load(f)
    print(file.keys())
    x_coords_u = file["x_coords"]
    y_coords_u = file["y_coords"]
    data_u_cut = file["signal"]
    noise_u_cut = file["noise"]
    good_vorbin_u = file["good_vorbin"]
    snr_target_u = file["snr_target"]
    binNum_u = file["binNum_u"]
    xNode_u = file["xNode"]
    yNode_u = file["yNode"]
    xBar_u = file["xBar"]
    yBar_u = file["yBar"]
    sn_u = file["sn"]
    nPixels_u = file["nPixels"]
    scale_u = file["scale"]
# 
shape_u = good_vorbin_u.shape
# img = ax.imshow(data_u_cut/noise_u_cut, origin="lower")
# binned_u = np.histogram2d(y_coords_u[good_vorbin_u], x_coords_u[good_vorbin_u], bins=[y_coords_u[good_vorbin_u], x_coords_u[good_vorbin_u]], weights=binNum_u)
binNum_arr = np.zeros(shape_u) * np.nan
for i, (x, y) in enumerate(zip(x_coords_u[good_vorbin_u], y_coords_u[good_vorbin_u])):
    binNum_arr[y, x] = binNum_u[i]

def calc_snr(signal, noise, func=np.nansum):
    """
    Calculates the overall signal-to-noise ratio of the input arrays.
    
    TODO: finish docstring
    
    Returns: snr, bin_signal, bin_noise
      snr :: float
        The total signal-to-noise ratio of the input data
    """
    bin_signal = func(signal)
    # Add noise in quadrature
    bin_noise = noise * noise
    bin_noise = func(bin_noise)
    bin_noise = np.sqrt(bin_noise)
    return bin_signal / bin_noise, bin_signal, bin_noise

binned_u_snr = np.zeros(shape_u) * np.nan
binned_u_signal = np.zeros(shape_u) * np.nan
binned_u_noise = np.zeros(shape_u) * np.nan
for i in np.unique(binNum_u):
    bin_idx = (binNum_arr == i)
    # if np.sum(bin_idx) != 1:
    #    print("binNum:", i, ". Elements in bin:", np.sum(bin_idx))
    binned_u_snr[bin_idx], binned_u_signal[bin_idx], binned_u_noise[bin_idx] = calc_snr(
        data_u_cut[bin_idx], noise_u_cut[bin_idx], func=np.nansum
    )
print(np.sum(np.isnan(binned_u_snr)))
fig, ax = plt.subplots()
img = ax.imshow(binned_u_snr, origin="lower")
cbar = fig.colorbar(img)
plt.show()

742182


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [125]:
from matplotlib.colors import LogNorm
plt.close("all")
%matplotlib widget
print("Min SNR:", np.nanmin(binned_u_snr))
nth_smallest = 7
print(f"{nth_smallest}th smallest SNR:", np.partition(np.unique(binned_u_snr[~np.isnan(binned_u_snr)]), nth_smallest)[nth_smallest - 1])
cutoff = np.percentile(binned_u_snr[~np.isnan(binned_u_snr)], 0.25)
print("Cutoff value:", cutoff)
# 
# 
# 
binned_u_masked = np.ma.masked_array(binned_u_snr, mask=binned_u_snr < cutoff).filled(fill_value=np.nan)
fig, ax = plt.subplots()
img = ax.imshow(binned_u_masked, origin="lower", norm=LogNorm())
cbar = fig.colorbar(img)
plt.show()
# print(np.sum(binned_u < 30), np.sum(binned_u < 30)/np.sum(~np.isnan(binned_u)))
# print(np.sum(binned_u < 29), np.sum(binned_u < 29)/np.sum(~np.isnan(binned_u)))
# print(np.sum(binned_u < 24), np.sum(binned_u < 24)/np.sum(~np.isnan(binned_u)))
fig, ax = plt.subplots()
ax.hist(binned_u.flatten(), bins=100)
ax.set_xlabel("SNR of Bin")
ax.set_ylabel("Frequency")
plt.show()
print(binNum_u.max())

Min SNR: 14.747917175292969
7th smallest SNR: 20.189800262451172
Cutoff value: 23.240521965026858


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

174108


In [133]:
binned_g_snr = np.zeros_like(data_g_cut) * np.nan
binned_g_signal = np.zeros_like(data_g_cut) * np.nan
binned_g_noise = np.zeros_like(data_g_cut) * np.nan
binned_i_snr = np.zeros_like(data_i_cut) * np.nan
binned_i_signal = np.zeros_like(data_i_cut) * np.nan
binned_i_noise = np.zeros_like(data_i_cut) * np.nan
binned_z_snr = np.zeros_like(data_z_cut) * np.nan
binned_z_signal = np.zeros_like(data_z_cut) * np.nan
binned_z_noise = np.zeros_like(data_z_cut) * np.nan
for i in np.unique(binNum_u):
    bin_idx = (binNum_arr == i)
    binned_g_snr[bin_idx], binned_g_signal[bin_idx], binned_g_noise[bin_idx] = calc_snr(data_g_cut[bin_idx], noise_g_cut[bin_idx], func=np.nansum)
    binned_i_snr[bin_idx], binned_i_signal[bin_idx], binned_i_noise[bin_idx] = calc_snr(data_i_cut[bin_idx], noise_i_cut[bin_idx], func=np.nansum)
    binned_z_snr[bin_idx], binned_z_signal[bin_idx], binned_z_noise[bin_idx] = calc_snr(data_z_cut[bin_idx], noise_z_cut[bin_idx], func=np.nansum)

In [124]:
plt.close("all")
%matplotlib widget
binned_g_masked = np.ma.masked_array(binned_g_snr, mask=binned_g_snr < cutoff).filled(fill_value=np.nan)
fig, ax = plt.subplots()
img = ax.imshow(binned_g_masked, origin="lower", norm=LogNorm())
cbar = fig.colorbar(img)
plt.show()
binned_i_masked = np.ma.masked_array(binned_i_snr, mask=binned_i_snr < cutoff).filled(fill_value=np.nan)
fig, ax = plt.subplots()
img = ax.imshow(binned_i_masked, origin="lower", norm=LogNorm())
cbar = fig.colorbar(img)
plt.show()
binned_z_masked = np.ma.masked_array(binned_z_snr, mask=binned_z_snr < cutoff).filled(fill_value=np.nan)
fig, ax = plt.subplots()
img = ax.imshow(binned_z_masked, origin="lower", norm=LogNorm())
cbar = fig.colorbar(img)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# 
# Save binned data
# 
binned_outfile = "/arc/home/IsaacCheng/coop_f2021/warmup/vcc792_allVorBinned_snr30.pkl"
# Load previous data (part 1)
with open(binned_outfile, "rb") as f:
    file = dill.load(f)
    x_coords_u = file["x_coords"]
    y_coords_u = file["y_coords"]
    binned_u_signal = file["u_binned_signal"]
    binned_u_noise = file["u_binned_noise"]
    binned_g_signal = file["g_binned_signal"]
    binned_g_noise = file["g_binned_noise"]
    binned_i_signal = file["i_binned_signal"]
    binned_i_noise = file["i_binned_noise"]
    binned_z_signal = file["z_binned_signal"]
    binned_z_noise = file["z_binned_noise"]
# Load previous data (part 2)
outfile_u = "/arc/home/IsaacCheng/coop_f2021/warmup/vcc792_u_snr30.pkl"
with open(outfile_u, "rb") as f:
    file = dill.load(f)
    good_vorbin_u = file["good_vorbin"]
    binNum_u = file["binNum"]
    nPixels = file["nPixels"]
#
binNum_arr = np.zeros_like(binned_u_signal) * np.nan
for i, (x, y) in enumerate(zip(x_coords_u[good_vorbin_u], y_coords_u[good_vorbin_u])):
    binNum_arr[y, x] = binNum_u[i]
# 
px_per_bin = np.zeros_like(binned_u_signal) * np.nan
for i in np.unique(binNum_u):
    bin_idx = (binNum_arr == i)
    px_per_bin[bin_idx] = nPixels[i]
with open(binned_outfile, "wb") as f:
    dill.dump(
        {
            "x_coords": x_coords_u,
            "y_coords": y_coords_u,
            "wcs": wcs_u_cut,
            "wcs_array_shape": x_coords_u.shape,
            "u_binned_signal": binned_u_signal,
            "u_binned_noise": binned_u_noise,
            "g_binned_signal": binned_g_signal,
            "g_binned_noise": binned_g_noise,
            "i_binned_signal": binned_i_signal,
            "i_binned_noise": binned_i_noise,
            "z_binned_signal": binned_z_signal,
            "z_binned_noise": binned_z_noise,
            "px_per_bin": px_per_bin
        },
        f,
    )
print("Pickled", binned_outfile)

Pickled /arc/home/IsaacCheng/coop_f2021/warmup/vcc792_allVorBinned_snr30.pkl


In [13]:
fig, ax = plt.subplots()
img = ax.imshow(px_per_bin, origin="lower", cmap="Blues")
fig.colorbar(img)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …